In [22]:
%matplotlib

import numpy as np
from matplotlib.patches import Circle, Wedge, Polygon
from matplotlib.collections import PatchCollection
import matplotlib.pyplot as plt
from numpy import arange
import math
import time

fig, ax = plt.subplots()

pathx = []
pathy = []
hpathx = []
hpathy = []
opathx = []
opathy = []
opathx2 = []
opathy2 = []
opath_deg = []
patches = []

a_path, = plt.plot(pathx, pathy)
a_hpath, = plt.plot(hpathx, hpathy, 'x')
a_opath, = plt.plot(opathx, opathy, '-r')
a_opath2, = plt.plot(opathx, opathy, '-m*')
selected_circle = None

def add_path(x, y):
    pathx.append(x)
    pathy.append(y)

def add_helper_point(x, y):
    hpathx.append(x)
    hpathy.append(y)

def add_opti_path(x, y):
    opathx.append(x)
    opathy.append(y)

def add_opti2_path(x, y):
    opathx2.append(x)
    opathy2.append(y)


circles = []

circles.append((0,0,0.4))
circles.append((1,7,2))
circles.append((9,9,2))
circles.append((6,2,1))
circles.append((10,0,2))
circles.append((13,7,0.4))

opath_deg = [0] * len(circles)

def recalc_optimal2():
    global tdist
    global tdist2
    
    global opathx2
    global opathy2
    
    opathx2 = []
    opathy2 = []
    
    add_opti2_path(*circles[0][:2])
    for i in range(len(opath_deg) - 2):
        i += 1
        if opath_deg[i] is not None:
            rad = math.radians(opath_deg[i])
            cx, cy, r = circles[i]
            x = cx + math.cos(rad) * r
            y = cy + math.sin(rad) * r
            add_opti2_path(x, y)
    add_opti2_path(*circles[-1][:2])

    tdist = 0
    tdist2 = 0

    for i in range(len(opathx) - 1):
        tdist += math.sqrt(pow(opathx[i] - opathx[i + 1], 2) + pow(opathy[i] - opathy[i + 1], 2))

    for i in range(len(opathx2) - 1):
        tdist2 += math.sqrt(pow(opathx2[i] - opathx2[i + 1], 2) + pow(opathy2[i] - opathy2[i + 1], 2))

def redraw(selected_circle = None, calc_circles = None):
    if (calc_circles == None):
        calc_circles = list(circles)
    
    global pathx
    global pathy
    global hpathx
    global hpathy
    global opathx
    global opathy
    global opathx2
    global opathy2
    global opath_deg
    global patches
    
    pathx = []
    pathy = []
    hpathx = []
    hpathy = []
    opathx = []
    opathy = []

    
    add_path(*circles[0][:2])
    for x, y, r in circles:
        add_path(x, y)


    #SOLVER
    def solve_point2circle(x, y, r, cx, cy, cr):
        v_x = x - cx
        v_y = y - cy
        magV = math.sqrt(pow(v_x, 2) + pow(v_y, 2))
        r_x = cx + v_x / magV * cr
        r_y = cy + v_y / magV * cr    

        v_x = ((v_x / magV))
        v_y = ((v_y / magV))    
        return r_x, r_y, v_x, v_y

    def solve_vector2circle(v_x, v_y, cx, cy, r):
        magV = math.sqrt(pow(v_x, 2) + pow(v_y, 2))
        r_x = cx + v_x / magV * r
        r_y = cy + v_y / magV * r    

        return r_x, r_y

    def get_opti_point(pindex, index, nindex):
        x, y, r = calc_circles[index]
        if pindex < 0 or nindex >= len(calc_circles):
            return x, y
        
        prev_x, prev_y, prev_r = calc_circles[pindex]
        next_x, nexy_y, next_r = calc_circles[nindex]

        rx, ry, vx1, vy1 = solve_point2circle(prev_x, prev_y, prev_r, x, y, r)
        add_helper_point(rx, ry)
        rx, ry, vx2, vy2 = solve_point2circle(next_x, nexy_y, next_r, x, y, r)
        add_helper_point(rx, ry)

        
        vx = x + vx1 + vx2
        vy = y + vy1 + vy2

#         if index == selected_circle:
#             add_opti2_path(x, y)
#             add_opti2_path(x + vx1, y + vy1)
#             add_opti2_path(vx, vy)
#             add_opti2_path(x + vx2, y + vy2)
#             add_opti2_path(x, y)
#             add_opti2_path(vx, vy)
#             add_opti2_path(None, None)


        rx, ry, __, __ = solve_point2circle(vx, vy, 0, x, y, r)
        add_helper_point(rx, ry)
        return rx, ry
        

    add_opti_path(*calc_circles[0][:2])

    for i in range(len(calc_circles) - 2):
        i = i + 1
        rx, ry = get_opti_point(i - 1, i, i + 1)
        add_opti_path(rx, ry)

    add_opti_path(*calc_circles[-1][:2])

#     #second pass
#     to_del = None
#     for i in range(len(calc_circles) - 2):
            
#         i = i + 1
#         prev_x = opathx[i - 1]
#         prev_y = opathy[i - 1]
#         x = opathx[i]
#         y = opathy[i]
#         next_x = opathx[i + 1]
#         next_y = opathy[i + 1]
#         cx, cy, r = calc_circles[i]
    
#         a = prev_y - next_y
#         b = next_x - prev_x
#         c = -(a * prev_x + b * prev_y)
        
#         dist = abs(a*cx + b*cy + c) / math.sqrt(pow(a,2) + pow(b,2))
        
#         if dist < r:
            
#             rx1, ry1 = get_opti_point(i - 2, i - 1, i + 1)
#             rx2, ry2 = get_opti_point(i - 1, i + 1, i + 2)
            
#             a = ry1 - ry2
#             b = rx2 - rx1
#             c = -(a * rx1 + b * ry1)

#             add_opti2_path(rx1, ry1)
#             add_opti2_path(rx2, ry2)            
            
#             dist = abs(a*cx + b*cy + c) / math.sqrt(pow(a,2) + pow(b,2))
#             #fig.canvas.set_window_title("2:%f" % dist)
            
#             if dist < r:
#                 to_del = i
#                 break
                
# #             ny = 100
# #             nx = (-b*ny-c)/a
#             add_opti2_path(nx, ny)
#             ny = -100
#             nx = (-b*ny-c)/a
#             add_opti2_path(nx, ny)
            

#             c = -(a * (prev_x - cx) + b * (prev_y - cy))
            
#             x0 = -a*c/(a*a+b*b)
#             y0 = -b*c/(a*a+b*b)
    
#             d = r*r - c*c/(a*a+b*b);
#             mult = math.sqrt (d / (a*a+b*b));

#             x1 = x0 + b * mult + cx;
#             x2 = x0 - b * mult + cx;
#             y1 = y0 - a * mult + cy;
#             y2 = y0 + a * mult + cy;            
            
#             #add_helper_point(x1, y1)
#             add_helper_point(x2, y2)
            
#             opathx[i] = x2
#             opathy[i] = y2
            
#     if to_del is not None:
#         del calc_circles[i]
#         redraw(selected_circle, calc_circles)

    for p in patches:
        p.remove()
    
    patches = []
    p = []
    i = 0
    for x, y, r in circles:
        if i == selected_circle:
            circle = Circle((x, y), r, alpha=0.6, ls='-', lw=1)
        else:
            circle = Circle((x, y), r, alpha=0.2, ls='-', lw=1)
            
        patches.append(ax.add_patch(circle))
        i += 1


    recalc_optimal2()

    
    fig.canvas.set_window_title("tot:%0.3f,%0.3f,%0.3f " % (tdist, tdist2, tdist2 - tdist))
    
    
    a_path.set_data(pathx, pathy)
    a_hpath.set_data(hpathx, hpathy)
    a_opath.set_data(opathx, opathy)
    a_opath2.set_data(opathx2, opathy2)
    fig.canvas.draw_idle()
#    fig.canvas.draw()

itter = [0]

def ev(event):
    x = event.xdata
    y = event.ydata
    if x is None or x is None:
        return
    
    global selected_circle
    global itter

    selected_circle = None
    for i in range(len(circles)):
        cx, cy, r = circles[i]
        if math.sqrt(pow(x - cx, 2) + pow(y - cy, 2)) <= r:
            selected_circle = i
    
    if event.button == 1 and selected_circle is not None:        # right mouse button
        cx, cy, r = circles[selected_circle]
        circles[selected_circle] = x, y, r
        redraw(selected_circle)
        itter = [0]
        
    if event.button == 2 and selected_circle is not None:        # middle mouse button
        
        start_time = time.process_time()
        while len(itter) < 5:
            improve = False
            for i in range(len(circles) - 2):
                i += 1
                min_dest = 1000000000000
                min_angle = None
                min_a = 100000000000000000

                last_angle = opath_deg[i]

                global tdist2

                if len(itter) == 1:
                    start = 0
                    end = 360
                    step = 10

                if len(itter) == 2:
                    start = last_angle - 18
                    end = last_angle + 18
                    step = 1

                if len(itter) == 3:
                    start = last_angle - 0.5
                    end = last_angle + 0.5
                    step = 0.1     
                    
                if len(itter) == 4:
                    start = last_angle - 0.05
                    end = last_angle + 0.05
                    step = 0.01                

                for angle in arange(start, end, step):
                    opath_deg[i] = angle
                    recalc_optimal2()

                    a = math.sqrt(pow(opathx2[i] - opathx2[i - 1], 2) + pow(opathy2[i] - opathy2[i - 1], 2))
                    b = math.sqrt(pow(opathx2[i] - opathx2[i + 1], 2) + pow(opathy2[i] - opathy2[i + 1], 2))
                    c = a + b + a / 100
                    
                    if min_dest > c:
                        min_dest = c
                        min_a = a
                        min_angle = angle


                if last_angle is None:
                    opath_deg[i] = min_angle  
                    improve = True
                    continue

                if abs(last_angle - min_angle) > step:
                    print(last_angle, min_angle)
                    opath_deg[i] = min_angle    
                    improve = True
                else:
                    opath_deg[i] = last_angle 

            itter[-1] += 1
            if not improve:
                itter.append(0)

            total = "%s elapsed %0.3fs" % (str(itter), time.process_time() - start_time)
            ax.set_title(total)

            redraw(None)      
            
            

       
        
def sc(event):
    global selected_circle
    
    if event.button == 'up' and selected_circle is not None:      
        if opath_deg[selected_circle] == None:
            opath_deg[selected_circle] = 0
        opath_deg[selected_circle] = (opath_deg[selected_circle] + 1 % 360)

        redraw(selected_circle)

    if event.button == 'down' and selected_circle is not None:      
        if opath_deg[selected_circle] == None:
            opath_deg[selected_circle] = 0
        opath_deg[selected_circle] = ((opath_deg[selected_circle] - 1 + 360)  % 360)

        redraw(selected_circle)

ax.set_xlim(-2, 16)
ax.set_ylim(-2, 16)

redraw(None)        


a_opath.figure.canvas.mpl_connect('motion_notify_event', ev)
a_opath.figure.canvas.mpl_connect('button_press_event', ev)
fig.canvas.mpl_connect('scroll_event', sc)

plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05)
plt.axis('equal')
plt.show()

Using matplotlib backend: TkAgg
0 310
0 230
0 30
0 110
310 313
30 33
313 312.7
230 229.8
33 32.8
312.7 312.8
312.8 312.77
32.8 32.82
110 110.04
32.82 32.81
229.8 222.8
110.04 117.04
312.77 313.17
32.81 32.31
117.04 116.74
313.17 313.57
32.31 31.81
313.57 313.77
222.8 223.0
313.77 313.72
223.0 222.97
31.81 31.84
313.72 313.7
313.7 320.7
222.97 216.97
31.84 32.84
320.7 321.1
216.97 216.77
32.84 32.64
116.74 116.64
321.1 321.3
321.3 321.33
216.77 216.75
32.64 32.6
116.64 116.68
32.6 90
116.68 320
216.75 213.75
90 80
320 325
321.33 321.63
213.75 213.95
80 79.5
325 324.6
79.5 79.1
324.6 324.7
321.63 321.59
213.95 213.97
324.7 324.66
79.1 79.11
79.11 60
324.66 300
60 66
300 304
213.97 214.37
66 65.5
304 304.2
214.37 214.77
321.59 321.54
214.77 214.8
65.5 65.48
304.2 304.24
321.54 321.51
304.24 290
65.48 56.48
290 286
214.8 215.2
56.48 56.28
286 286.1
215.2 215.4
321.51 321.46
215.4 215.37
56.28 56.27
56.27 160
286.1 260
160 162
260 264
215.37 215.17
162 162.4
264 264.4
215.17 215.37
162.4 16